In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import ReduceLROnPlateau
import os
from time import time

# Configuration
os.makedirs("results", exist_ok=True)
IMG_SIZE = (64, 64)
BATCH_SIZE = 128  # Augmenté pour accélérer l'entraînement
EPOCHS = 30      # Réduit grâce aux optimisations

def build_optimized_cae(input_shape=(64, 64, 1)):
    """Autoencodeur avec optimisation de l'architecture"""
    input_img = Input(shape=input_shape)
    
    # Encoder optimisé
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)
    
    # Decoder
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer=Nadam(learning_rate=0.001), loss='mse')
    return autoencoder

def load_and_preprocess(dataset_path):
    """Chargement avec vérification et preprocessing accéléré"""
    data = np.load(dataset_path).astype('float32') / 255.0
    if len(data.shape) == 3:
        data = data[..., np.newaxis]
    return data

def train_with_progress(model, X_train, dataset_name):
    """Entraînement avec callback de progression"""
    start_time = time()
    
    # Callback pour réduire le LR dynamiquement
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=1e-5)
    
    history = model.fit(
        X_train, X_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        shuffle=True,
        callbacks=[reduce_lr],
        verbose=1
    )
    
    # Sauvegarde rapide du modèle
    model.save(f"results/cae_{dataset_name}.h5", save_format='h5')
    
    print(f"\nEntraînement terminé en {(time()-start_time)/60:.1f} minutes")
    
    # Courbe de loss
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.title('Evolution de la Loss pendant l\'entraînement')
    plt.ylabel('MSE Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.savefig(f"results/{dataset_name}_training_curve.png")
    plt.close()

def visualize_reconstructions(model, X_test, dataset_name, n=10):
    """Visualisation améliorée des reconstructions"""
    samples = X_test[:n]
    reconstructions = model.predict(samples, batch_size=BATCH_SIZE)
    
    plt.figure(figsize=(20, 6))
    for i in range(n):
        # Image originale
        plt.subplot(3, n, i+1)
        plt.imshow(samples[i].squeeze(), cmap='gray')
        plt.title("Original" if i == 0 else "", pad=10)
        plt.axis('off')
        
        # Reconstruction
        plt.subplot(3, n, i+n+1)
        plt.imshow(reconstructions[i].squeeze(), cmap='gray')
        plt.title("Reconstruite" if i == 0 else "", pad=10)
        plt.axis('off')
        
        # Différence
        plt.subplot(3, n, i+2*n+1)
        plt.imshow(np.abs(samples[i] - reconstructions[i]).squeeze(), cmap='hot')
        plt.title("Différence" if i == 0 else "", pad=10)
        plt.axis('off')
    
    plt.suptitle(f"Comparaison Original/Reconstruction - {dataset_name}", y=1.02)
    plt.tight_layout()
    plt.savefig(f"results/{dataset_name}_reconstructions.png", bbox_inches='tight')
    plt.close()

def detect_and_visualize_anomalies(model, X_test, dataset_name):
    """Détection et visualisation des anomalies"""
    # Calcul des erreurs
    reconstructions = model.predict(X_test, batch_size=BATCH_SIZE)
    errors = np.mean(np.square(X_test - reconstructions), axis=(1, 2, 3))
    
    # Seuil automatique (95e percentile)
    threshold = np.percentile(errors, 95)
    
    # Top 5 anomalies
    top_anomalies = np.argsort(errors)[-5:][::-1]
    
    # Visualisation
    plt.figure(figsize=(15, 9))
    for i, idx in enumerate(top_anomalies):
        # Original
        plt.subplot(3, 5, i+1)
        plt.imshow(X_test[idx].squeeze(), cmap='gray')
        plt.title(f"Original {i+1}\nErr: {errors[idx]:.4f}")
        plt.axis('off')
        
        # Reconstruction
        plt.subplot(3, 5, i+6)
        plt.imshow(reconstructions[idx].squeeze(), cmap='gray')
        plt.title("Reconstruction")
        plt.axis('off')
        
        # Différence
        plt.subplot(3, 5, i+11)
        plt.imshow(np.abs(X_test[idx] - reconstructions[idx]).squeeze(), cmap='hot')
        plt.title("Différence")
        plt.axis('off')
    
    plt.suptitle(f"Top 5 Anomalies Détectées - {dataset_name}\nSeuil: {threshold:.4f}", y=1.05)
    plt.tight_layout()
    plt.savefig(f"results/{dataset_name}_top_anomalies.png", bbox_inches='tight')
    plt.close()
    
    return threshold

def main():
    datasets = {
        'UCSDped1': {'train': 'UCSDped1_train.npy', 'test': 'UCSDped1_test.npy'},
        'UCSDped2': {'train': 'UCSDped2_train.npy', 'test': 'UCSDped2_test.npy'}
    }
    
    for name, paths in datasets.items():
        print(f"\n=== Traitement de {name} ===")
        
        try:
            # Chargement accéléré
            print("Chargement des données...")
            X_train = load_and_preprocess(paths['train'])
            X_test = load_and_preprocess(paths['test'])
            
            # Construction et entraînement
            print("Construction du modèle...")
            model = build_optimized_cae()
            
            print("Début de l'entraînement...")
            train_with_progress(model, X_train, name)
            
            # Visualisations
            print("Génération des visualisations...")
            visualize_reconstructions(model, X_test, name)
            threshold = detect_and_visualize_anomalies(model, X_test, name)
            
            print(f"\nRésultats pour {name}:")
            print(f"- Seuil automatique: {threshold:.4f}")
            print(f"- Visualisations sauvegardées dans /results")
            
        except Exception as e:
            print(f"Erreur sur {name}: {str(e)}")

if __name__ == "__main__":
    main()


=== Traitement de UCSDped1 ===
Chargement des données...
Construction du modèle...
Début de l'entraînement...
Epoch 1/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 19s 302ms/step - loss: 0.1057 - learning_rate: 0.0010
Epoch 2/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 16s 304ms/step - loss: 9.7655e-06 - learning_rate: 0.0010
Epoch 3/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 17s 307ms/step - loss: 4.7167e-06 - learning_rate: 0.0010
Epoch 4/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 19s 271ms/step - loss: 3.5128e-06 - learning_rate: 0.0010
Epoch 5/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 16s 288ms/step - loss: 3.1483e-06 - learning_rate: 0.0010
Epoch 6/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 16s 290ms/step - loss: 3.0180e-06 - learning_rate: 2.0000e-04
Epoch 7/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 16s 287ms/step - loss: 2.9966e-06 - learning_rate: 2.0000e-04
Epoch 8/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 15s 286ms/step - loss: 2.9789e-06 - learning_rate: 2.0000e-04
Epoch 9/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 16s 287ms/step - loss: 2.9635e-06 - learning_rate: 4.0000e-05
Epoch 10/30
5


Entraînement terminé en 8.1 minutes
Génération des visualisations...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 7s 118ms/step

Résultats pour UCSDped1:
- Seuil automatique: 0.0000
- Visualisations sauvegardées dans /results

=== Traitement de UCSDped2 ===
Chargement des données...
Construction du modèle...
Début de l'entraînement...
Epoch 1/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 277ms/step - loss: 0.1819 - learning_rate: 0.0010
Epoch 2/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 306ms/step - loss: 3.2780e-04 - learning_rate: 0.0010
Epoch 3/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 305ms/step - loss: 2.3907e-05 - learning_rate: 0.0010
Epoch 4/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 303ms/step - loss: 1.5809e-05 - learning_rate: 0.0010
Epoch 5/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 319ms/step - loss: 1.3886e-05 - learning_rate: 0.0010
Epoch 6/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 247ms/step - loss: 1.2645e-05 - learning_rate: 0.0010
Epoch 7/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 304ms/step - loss: 1.1729e-05 - le